In [2]:
import torch
import time
import timm
import torchvision.transforms as transforms
from PIL import Image

# Load pre-trained Swin Transformer models
model_384 = timm.create_model('swin_base_patch4_window12_384_in22k', pretrained=False)
model_224 = timm.create_model('swin_base_patch4_window7_224_in22k', pretrained=False)


/home/yuxin/miniconda3/envs/mmpretrain/lib/python3.11/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swin_base_patch4_window12_384_in22k to current swin_base_patch4_window12_384.ms_in22k.
  model = create_fn(
/home/yuxin/miniconda3/envs/mmpretrain/lib/python3.11/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swin_base_patch4_window7_224_in22k to current swin_base_patch4_window7_224.ms_in22k.
  model = create_fn(


In [3]:
model_224.eval()

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate='none')
            (

In [4]:
model_384.eval()

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate='none')
            (

In [5]:
for name, param in model_384.state_dict().items():
    if name in model_224.state_dict():
        if param.size() == model_224.state_dict()[name].size():
            model_224.state_dict()[name].copy_(param)
        else:
            print('size mismatch: {}, {}'.format(name, param.size()))

size mismatch: layers.0.blocks.0.attn.relative_position_bias_table, torch.Size([529, 4])
size mismatch: layers.0.blocks.1.attn.relative_position_bias_table, torch.Size([529, 4])
size mismatch: layers.1.blocks.0.attn.relative_position_bias_table, torch.Size([529, 8])
size mismatch: layers.1.blocks.1.attn.relative_position_bias_table, torch.Size([529, 8])
size mismatch: layers.2.blocks.0.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch: layers.2.blocks.1.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch: layers.2.blocks.2.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch: layers.2.blocks.3.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch: layers.2.blocks.4.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch: layers.2.blocks.5.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch: layers.2.blocks.6.attn.relative_position_bias_table, torch.Size([529, 16])
size mismatch:

In [7]:
ckpt_path = '/mnt/hd0/project_large_files/bca_grading/suqh/swin_multi_task/2023_12_15_18:17:54/model-34.pth'
ckpt = torch.load(ckpt_path)

In [13]:
'module.base_model.head.fc.bias' in ckpt

True

In [15]:
for name, param in model_224.state_dict().items():
    if f'module.base_model.{name}' in ckpt:
        if param.size() == ckpt[f'module.base_model.{name}'].size():
            print('load {}'.format(name))
            model_224.state_dict()[name].copy_(param)
        else:
            print('size mismatch: {}, {}'.format(name, param.size()))

load patch_embed.proj.weight
load patch_embed.proj.bias
load patch_embed.norm.weight
load patch_embed.norm.bias
load layers.0.blocks.0.norm1.weight
load layers.0.blocks.0.norm1.bias
load layers.0.blocks.0.attn.qkv.weight
load layers.0.blocks.0.attn.proj.weight
load layers.0.blocks.0.attn.proj.bias
load layers.0.blocks.0.norm2.weight
load layers.0.blocks.0.norm2.bias
load layers.0.blocks.0.mlp.fc1.weight
load layers.0.blocks.0.mlp.fc1.bias
load layers.0.blocks.0.mlp.fc2.weight
load layers.0.blocks.0.mlp.fc2.bias
load layers.0.blocks.1.norm1.weight
load layers.0.blocks.1.norm1.bias
load layers.0.blocks.1.attn.qkv.weight
load layers.0.blocks.1.attn.proj.weight
load layers.0.blocks.1.attn.proj.bias
load layers.0.blocks.1.norm2.weight
load layers.0.blocks.1.norm2.bias
load layers.0.blocks.1.mlp.fc1.weight
load layers.0.blocks.1.mlp.fc1.bias
load layers.0.blocks.1.mlp.fc2.weight
load layers.0.blocks.1.mlp.fc2.bias
size mismatch: layers.1.downsample.norm.weight, torch.Size([512])
size mismatc